# Road Normalization and Sector Mapping
# 
# This notebook contains the road mapping and normalization logic used to create
# flooded_roads_phase1.csv from the original MMDA flooded roads data.
# 
# The process includes:
# 1. Loading and cleaning the raw MMDA data
# 2. Text normalization for location names
# 3. Road sector mapping (EDSA, C5, major roads)
# 4. Filtering to phase 1 roads
# 5. Exporting the processed data


In [ ]:
import pandas as pd
import re
import os


In [ ]:
# Load the original MMDA data
df = pd.read_csv("../data/interim/mmda_flooded_roads.csv", on_bad_lines="skip")
print(f"Original MMDA data shape: {df.shape}")
df.head()


In [ ]:
# Data overview
df.info()


In [ ]:
# Basic statistics
df.describe()


In [ ]:
# Convert datetime columns
df["datetime"] = pd.to_datetime(
    df["Date"].astype(str) + " " + df["Time"].astype(str),
    errors="coerce"
)
df.drop(columns=["Date", "Time"], inplace=True)

print("Datetime conversion completed!")
df.info()


In [ ]:
# Check unique cities
df["City"].unique()


In [ ]:
# Handle missing passability data
df["Passability"] = df["Passability"].fillna(
    df["Flood Type/Depth"].apply(
        lambda x: "Passable to all vehicles" if x == "Subsided" else "Unknown"
    )
)
df["Passability"].unique()


In [ ]:
# Check number of unique locations
df["Location"].nunique()


In [ ]:
# Show sample locations
df["Location"].unique()


In [ ]:
# Helper function for searching roads
def search_roads(df, search_str, column):
    mask_search = df[column].str.contains(search_str, case=False, na=False)
    return df[mask_search]


In [ ]:
# ===== TEXT NORMALIZATION =====
def normalize_location(text):
    """Standardize location strings before matching"""
    text = text.upper()
    # Remove directions
    text = re.sub(r'\b(NB|SB|EB|WB|NORTHBOUND|SOUTHBOUND|EASTBOUND|WESTBOUND)\b', '', text)
    # Standardize abbreviations
    text = text.replace("AVE.", "AVENUE").replace("AVE", "AVENUE")
    text = text.replace("EXT.", "EXTENSION").replace("RD.", "ROAD").replace("RD", "ROAD")
    # Remove service descriptors
    text = re.sub(r"SERVICE ROAD|AFTER FLYOVER|BEFORE FLYOVER|TUNNEL|INTERSECTION", "", text)
    # Collapse spaces
    text = re.sub(r"\s+", " ", text).strip()
    return text

# ===== EDSA SEGMENTATION =====
def map_edsa_segment(location):
    """Map EDSA locations into defined subsegments"""
    loc = location.upper()
    edsa_segments = {
        'EDSA_CALOOCAN': ['MONUMENTO', 'BALINTAWAK'],
        'EDSA_QC_NORTH': ['ROOSEVELT', 'NORTH AVENUE', 'MUÑOZ', 'MUNOZ', 'DARIO', 'OLIVEROS', 'BAGONG BARRIO'],
        'EDSA_QC_CENTRAL': ['QUEZON AVENUE', 'CENTRIS', 'CENTRAL TERMINAL'],
        'EDSA_QC_SOUTH': ['KAMUNING', 'TIMOG', 'SCOUT', 'MOTHER IGNACIA', 'WEST AVENUE', 'MAIN AVENUE', 'WHITEPLAINS', 'P. TUAZON'],
        'EDSA_QC_BOUNDARY': ['AURORA', 'SANTOLAN', 'AGUINALDO', 'LIBERTY'],
        'EDSA_SAN_JUAN': ['SAN JUAN'],
        'EDSA_MANDALUYONG': ['ORTIGAS', 'SHAW', 'MEGAMALL', 'BONI', 'WACK WACK'],
        'EDSA_MAKATI': ['AYALA', 'BUENDIA', 'MAGALLANES', 'GUADALUPE'],
        'EDSA_PASAY': ['TAFT', 'ROXAS', 'TRAMO', 'MALL OF ASIA', 'MOA']
    }
    for segment, keywords in edsa_segments.items():
        for kw in keywords:
            if kw in loc:
                return segment
    return "EDSA_GENERAL"

# ===== C5 SEGMENTATION (includes Katipunan) =====
def map_c5_segment(location):
    """Map C5/Katipunan locations into subsegments"""
    loc = location.upper()
    c5_segments = {
        'C5_NORTH': ['NLEX', 'HARBOR', 'KARUHATAN', 'VALENZUELA'],
        'C5_CONGRESSIONAL': ['CONGRESSIONAL', 'BAHAY TORO', 'CULIAT', 'PASONG TAMO'],
        'C5_LUZON': ['LUZON', 'COMMONWEALTH', 'FLYOVER'],
        'C5_TANDANG_SORA': ['TANDANG SORA', 'U.P.', 'DILIMAN'],
        'C5_KATIPUNAN': ['KATIPUNAN', 'LOYOLA', 'MIRIAM', 'ATENEO', 'PROJECT 4', 'PANSOL'],
        'C5_BONNY_SERRANO': ['BONNY', 'SERRANO', 'LIBIS', 'TUNNEL'],
        'C5_E_RODRIGUEZ': ['E. RODRIGUEZ', 'EULOGIO'],
        'C5_GARCIA': ['CARLOS GARCIA', 'C.P. GARCIA', 'BGC', 'TAGUIG'],
        'C5_EXTENSION': ['EXTENSION', 'SLEX', 'NAIA', 'AIRPORT', 'PARAÑAQUE', 'LAS PIÑAS']
    }
    for segment, keywords in c5_segments.items():
        for kw in keywords:
            if kw in loc:
                return segment
    return "C5_GENERAL"


In [ ]:
# ===== GENERAL ROAD MAPPING =====
def map_road_segment(location, city):
    loc = normalize_location(location)
    
    # Special handling: EDSA
    if "EDSA" in loc:
        return map_edsa_segment(loc)
    
    # Special handling: C5 or Katipunan
    if "C5" in loc or "KATIPUNAN" in loc:
        return map_c5_segment(loc)
    
    # Simple rules (examples — expand as needed)
    if "ELLIPTICAL" in loc:
        return "EAST AVENUE"
    if "C3" in loc or "CONNECTOR" in loc:
        return "C3 ROAD"
    if "N.S AMORANTO" in loc or "NS AMORANTO" in loc:
        return "N.S. AMORANTO AVENUE"
    if "ESPANA" in loc or "ESPAÑA" in loc:
        return "ESPAÑA BOULEVARD"
    if "TAFT" in loc or "QUIRINO STATION" in loc:
        return "TAFT AVENUE"
    if ("DR. A. SANTOS" in loc or "A. SANTOS" in loc or "SM SUCAT" in loc) and city == "Parañaque City":
        return "DR. SANTOS AVENUE"
    if "JOSE ABAD SANTOS" in loc or "JAS" in loc:
        return "ABAD SANTOS AVENUE"
    if "COMMONWEALTH" in loc:
        return "COMMONWEALTH AVENUE"
    if "ARANETA" in loc:
        return "ARANETA AVENUE"
    if "MC ARTHUR" in loc or "MCARTHUR" in loc:
        return "MACARTHUR HIGHWAY"
    if "ROXAS" in loc:
        return "ROXAS BOULEVARD"
    if "QUIRINO AVE" in loc:
        return "QUIRINO AVENUE"
    if "RIZAL AVE" in loc:
        return "RIZAL AVENUE"
    if "PEDRO GIL" in loc:
        return "PEDRO GIL Street"
    if "QUEZON AVE" in loc:
        return "QUEZON AVENUE"
    if "BONIFACIO AVE" in loc:
        return "ANDRES BONIFACIO AVENUE"
    if "RECTO" in loc:
        return "RECTO AVENUE"
    if "SAMSON ROAD" in loc:
        return "SAMSON ROAD"
    if "DEL PILAR" in loc:
        return "DEL PILAR STREET"
    if "DON BASILIO BAUTISTA" in loc:
        return "DON BASILIO BAUTISTA BOULEVARD"
    if "ANDREW" in loc:
        return "ANDREWS AVENUE"
    if "GOV. PASCUAL" in loc:
        return "GOVERNOR PASCUAL AVENUE"
    if "OSMENIA" in loc or "OSMEÑA" in loc:
        return "OSMEÑA HIGHWAY"
    if "PADRE FAURA" in loc:
        return "PADRE FAURA STREET"
    if "MIA" in loc:
        return "MIA Road"
    if "ZAPOTE" in loc:
        return "ALABANG-ZAPOTE ROAD"
    if "MAGALLANES" in loc:
        return "MAGALLANES INTERCHANGE"
    if "PALANCA" in loc:
        return  "CARLOS PALANCA STREET"
    if "PNU TO AYALA" in loc or "AYALA BLVD"  in loc:
        return "AYALA BOULEVARD"
    if "UN. AVE" in loc:
        return "UN AVENUE"
    if "IN FRONT" in loc or "GATE" in loc or "BRGY" in loc or "CITY HALL" in loc:
        return None
    
    # Default: cleaned-up location
    global x
    x += 1
    print(loc)
    return loc

# ===== APPLY TO DATAFRAME =====
def add_road_sector(df):
    df["Road_Sector"] = df.apply(
        lambda row: map_road_segment(row["Location"], row["City"]),
        axis=1
    )
    return df

x = 0
df2 = add_road_sector(df)
print(f"Unmapped locations: {x}")
print(f"Unique Road Segments: {df2['Road_Sector'].nunique()}")
df2["Road_Sector"].unique()


In [ ]:
# Test search function
search_roads(df2, search_str="PALANCA ", column="Road_Sector")


In [ ]:
# Identify rare roads (less than 5 occurrences)
rare_roads = df2["Road_Sector"].value_counts()[lambda x: x < 5].index
df2_filtered = df2[df2["Road_Sector"].isin(rare_roads)]
print(f"Rare roads (less than 5 occurrences): {len(rare_roads)}")
df2_filtered


In [ ]:
# Define Phase 1 roads (major roads and segments)
phase1_roads = [
    # EDSA Segments
    "EDSA_CALOOCAN",
    "EDSA_QC_NORTH",
    "EDSA_QC_CENTRAL",
    "EDSA_QC_SOUTH",
    "EDSA_QC_BOUNDARY",
    "EDSA_MANDALUYONG",
    "EDSA_MAKATI",
    "EDSA_PASAY",

    # C5 Segments
    "C5_CONGRESSIONAL",
    "C5_LUZON",
    "C5_KATIPUNAN",
    "C5_BONNY_SERRANO",
    "C5_GARCIA",
    "C5_EXTENSION",

    # Major Radials & City Arteries
    "COMMONWEALTH AVENUE",
    "ESPAÑA BOULEVARD",
    "QUEZON AVENUE",
    "AURORA BOULEVARD",
    "RECTO AVENUE",
    "MACARTHUR HIGHWAY",

    # South & Coastal Roads
    "TAFT AVENUE",
    "ROXAS BOULEVARD",
    "DR. SANTOS AVENUE",
    "QUIRINO AVENUE",
    "OSMEÑA HIGHWAY"
]

print(f"Phase 1 roads defined: {len(phase1_roads)}")
print("Phase 1 roads:")
for road in phase1_roads:
    print(f"  - {road}")


In [ ]:
# Filter to Phase 1 roads only
df_phase1 = df2[df2["Road_Sector"].isin(phase1_roads)].copy()

# Reset index for cleaner data
df_phase1.reset_index(drop=True, inplace=True)

print(f"Phase 1 data shape: {df_phase1.shape}")
print(f"Original data: {len(df2)} records")
print(f"Phase 1 data: {len(df_phase1)} records")
print(f"Reduction: {((len(df2) - len(df_phase1)) / len(df2) * 100):.1f}%")

df_phase1.info()


In [ ]:
# Show distribution of road sectors in Phase 1 data
print("Road sector distribution in Phase 1 data:")
sector_counts = df_phase1["Road_Sector"].value_counts()
print(sector_counts)

print(f"\nUnique road sectors in Phase 1: {df_phase1['Road_Sector'].nunique()}")
print(f"Unique cities in Phase 1: {df_phase1['City'].nunique()}")
print(f"Cities: {df_phase1['City'].unique()}")


In [ ]:
# Export Phase 1 data
output_path = "../data/interim/flooded_roads_phase1.csv"
df_phase1.to_csv(output_path, index=False)
print(f"Phase 1 data exported to: {output_path}")
print(f"Final dataset shape: {df_phase1.shape}")

# Show sample of final data
print("\nSample of Phase 1 data:")
df_phase1.head(10)


In [ ]:
# Summary statistics
print("=== ROAD NORMALIZATION SUMMARY ===")
print(f"Original MMDA records: {len(df)}")
print(f"After road sector mapping: {len(df2)}")
print(f"Phase 1 filtered records: {len(df_phase1)}")
print(f"Unique road sectors mapped: {df2['Road_Sector'].nunique()}")
print(f"Phase 1 road sectors: {df_phase1['Road_Sector'].nunique()}")
print(f"Unmapped locations: {x}")
print(f"Date range: {df_phase1['datetime'].min()} to {df_phase1['datetime'].max()}")
print(f"\nData successfully processed and exported to {output_path}")
